# Data Science for Medicine and Public Health: Applied Statistical Learning
**National Preventive Medicine Residency Program**<br>
**Resident-Led Session, 29 Mar 2018**<br>

## Sample Code: Lung Cancer Diagnosis

You are given a dataset of 1105 chest X-ray (CXR) images, each described by 102 attributes and a label, where 1 indicates malignant lung cancer, and 0 indicates benign tumour. Your task is to build a classification model to predict whether a given CXR shows a malignant or benign tumour.

In [0]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

## Data Preparation

In [2]:
# load and inspect the data
data_original = pd.read_csv('lungsample.csv')

#df_dict = pd.read_excel('2010.xlsx',sheet_name=None)
#data = pd.concat(df_dict.values(), axis=0)

data = pd.read_csv('2010.xlsx - None.csv')
data.head()


FileNotFoundError: ignored

In [0]:
print(data_original)
data_original.shape, data.shape

            v0        v1        v2        v3        v4        v5        v6  \
0    -0.164889  0.401110  0.555741  0.083018  1.248687  0.419196 -0.132630   
1     0.466852 -0.362214  0.290673  0.358680  0.288683  0.169915 -0.130182   
2    -0.385892 -1.351414  1.010566 -1.212319 -1.914623 -1.130040 -0.014538   
3    -0.188841  0.987212 -0.480216  0.162236  0.687001 -0.090172 -0.113926   
4     0.805155 -1.343913  0.068894 -1.066623 -1.697915 -1.105640 -0.004081   
5    -0.902733 -2.883427  0.271698 -1.306577 -1.756070 -1.099451  0.514924   
6    -0.916640 -1.179506 -0.126200 -0.779572  1.140494 -0.976118 -0.114371   
7    -0.088993  0.448359 -0.556119 -0.694098 -0.785462 -0.731451 -0.059460   
8     0.777437 -0.377060  0.356495 -0.716436 -1.570038 -1.096894  0.017807   
9     1.307274  0.211126 -0.183720 -0.943134 -0.340002 -0.974623 -0.039493   
10   -0.263579 -0.278448  0.223072 -0.118365  0.070949  0.105817 -0.108845   
11    0.442816 -0.274905  0.007223  1.031794 -0.591029 -0.976753

((1104, 103), (438, 54))

In [0]:
# prepare predictor and response arrays from data
x = data.values[:,2:53]
y = data.values[:,49:53]
#y = data.values[:,53]
print ('x:', x.shape)
print ('y:', y.shape)



x: (438, 51)
y: (438, 4)


<i>I will now split the data into training (70%) and testing (30%) sets, and check that their dimensions are correct.</i>

In [0]:
# split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.7, random_state=42)

# sanity check
print ('Train set: ', x_train.shape)
print ('Test set: ', x_test.shape)
print ('Train benign: {}, Train malignant: {}'.format(len(y_train[y_train==0]), len(y_train[y_train==1])))
print ('Test benign: {}, Test malignant: {}'.format(len(y_test[y_test==0]), len(y_test[y_test==1])))

Train set:  (131, 51)
Test set:  (307, 51)
Train benign: 30, Train malignant: 4
Test benign: 74, Test malignant: 8


## Model Building

<i>I will now proceed to train 2 classifiers using random forest and support vector machine, and evaluate their performance using test accuracy, sensitivity and specificity.</i>

In [0]:
# function for computing the overall test accuracy, sensitivity and specificity of a given model
score = lambda model, x_test, y_test: pd.Series([model.score(x_test, y_test), 
                                                 model.score(x_test[y_test==0], y_test[y_test==0]),
                                                 model.score(x_test[y_test==1], y_test[y_test==1])],
                                                index=['Overall test accuracy', 'Accuracy on benign (Specificity)', 'Accuracy on malignant (Sensitivity)'])

In [0]:
# random forest
rf = RandomForestClassifier()
rf.fit(x_train, y_train)
rf_scores = score(rf, x_test, y_test)

# svm
svm = SVC()
svm.fit(x_train, y_train)
svm_scores = score(svm, x_test, y_test)

# present scores in a dataframe
score_df = pd.DataFrame({'Random Forest': rf_scores, 'SVM': svm_scores})
score_df

ValueError: Unknown label type: 'unknown'

## Parameter Tuning

<i>Now, I will try to tune the parameters of the random forest and SVM models, and see if they can perform better. For random forest, I will use 5-fold cross-validation to find the best max depth of trees. For SVM, I will use 5-fold cross-validation to find the best regularization parameter C.</i>

In [0]:
# tune random forest
parameters = {'max_depth': range(1, 10)}
rf = RandomForestClassifier(class_weight='balanced', n_estimators=30)
rf_tuned = GridSearchCV(rf, parameters, cv=5)
rf_tuned.fit(x_train, y_train)
print ('Test accuracy for random forest model with {}: {}'.format(rf_tuned.best_params_, score(rf_tuned, x_test, y_test)[0]))

In [0]:
# tune svm
parameters = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}
svm = SVC(class_weight='balanced', kernel='linear')
svm_tuned = GridSearchCV(svm, parameters, cv=5)
svm_tuned.fit(x_train, y_train)
print ('Test accuracy for SVM model with {}: {}'.format(svm_tuned.best_params_, score(svm_tuned, x_test, y_test)[0]))

## Model Evaluation

<i>Now, I will plot the Receiver Operating Characteristic (ROC) curves for the 2 models to compare their areas under the curve.</i>

In [0]:
fpr = dict()
tpr = dict()
roc_auc = dict()

# random forest
rf = RandomForestClassifier(class_weight='balanced', n_estimators=30, max_depth=6)
rf_probas = rf.fit(x_train, y_train).predict_proba(x_test)
fpr["RF"], tpr["RF"], _ = roc_curve(y_test, rf_probas[:, 1])
roc_auc["RF"] = auc(fpr["RF"], tpr["RF"])

# svm
svm = SVC(class_weight='balanced', kernel='linear', C=0.1)
svm_probas = svm.fit(x_train, y_train).decision_function(x_test)
fpr["SVM"], tpr["SVM"], _ = roc_curve(y_test, svm_probas)
roc_auc["SVM"] = auc(fpr["SVM"], tpr["SVM"])

# plot
plt.figure()
plt.plot(fpr["RF"], tpr["RF"], color='darkviolet', lw=2, label='RF (AUC = %0.2f)' % roc_auc["RF"])
plt.plot(fpr["SVM"], tpr["SVM"], color='darkorange', lw=2, label='SVM (AUC = %0.2f)' % roc_auc["SVM"])
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves')
plt.legend(loc="lower right")
plt.show()

<i>I will now visualize the top 5 most predictive features and their relative importance in the random forest model.</i>

In [0]:
# obtain feature importance list and sort
importances = rf.feature_importances_
indices = np.argsort(importances)
top5_indices = indices[-5:]

# extract gene names
features = data.columns

# plot
plt.title('Top 5 Predictive Features')
plt.barh(range(len(top5_indices)), importances[top5_indices], color='blue', align='center')
plt.yticks(range(len(top5_indices)), features[top5_indices])
plt.xlabel('Relative Importance')
plt.show()